In [1]:
import pandas as pd

In [40]:
#데이터 불러오기
df1 = pd.read_csv('./20sight_new.csv')
df2 = pd.read_csv('./31naver_new.csv')
df3 = pd.read_csv('./31trip_new.csv')

#concat
df = pd.concat([df1, df2, df3])

#필요없는 열 제거
df.drop(columns = ['Unnamed: 0', 'bridge'], inplace=True)

df.head()

,name,date,reviews,stars,lng,lat
0,공룡공원,2020-08,굿,3,128.603395,35.829925
1,공룡공원,2020-08,굿,3,128.603395,35.829925
2,국채보상운동기념공원,2020-08,좋아요,5,128.602750,35.869007
3,국채보상운동기념공원,2020-06,주차장 이용했습니다아,5,128.602750,35.869007
4,국채보상운동기념공원,2020-02,대구를 상징하는 좋아요,3,128.602750,35.869007


In [41]:
#관광지명 통일화
D_name = {'건들바위(대구입암)' : '건들바위', 
    '김광석 다시 그리기 길' : '김광석 길', 
    '김광석스토리하우스' : '김광석 길'}

df['name2'] = df['name'].map(D_name)
df['name2'] = df['name2'].fillna(df['name'])


#위치정보 통일화
L_name = list(df.name2.unique())

for name in L_name :
    #관광지 하나만 담은 df
    df_test = df[df['name2'] == name]

    #평균 위경도 계산
    lng = round(df_test.lng.mean(), 6)
    lat = round(df_test.lat.mean(), 6)
    
    #df에 기입
    df.loc[df['name2'] == name, 'lng'] = lng
    df.loc[df['name2'] == name, 'lat'] = lat

#열 위치 바꾸기
df.drop(columns='name', inplace=True)
df = df[['name2', 'date', 'reviews', 'stars', 'lng', 'lat']]
df.columns = ['name', 'date', 'reviews', 'stars', 'lng', 'lat']

df.head()

,name,date,reviews,stars,lng,lat
0,공룡공원,2020-08,굿,3,128.603395,35.829925
1,공룡공원,2020-08,굿,3,128.603395,35.829925
2,국채보상운동기념공원,2020-08,좋아요,5,128.602750,35.869007
3,국채보상운동기념공원,2020-06,주차장 이용했습니다아,5,128.602750,35.869007
4,국채보상운동기념공원,2020-02,대구를 상징하는 좋아요,3,128.602750,35.869007


In [43]:
#관광지 카테고리화
#df_name = df.groupby('name').count().reset_index(drop=False)
#df_name.name.to_csv('sight_code2.csv', encoding='cp949')

#직접 대구 관광명소 리스트를 네이버 플레이스 분류와 한국관광공사 관광자원 분류기준을 참고하여 카테고리화 시킴
df_code = pd.read_csv('./sight_code2.csv', encoding='cp949')
df_code

,name,cate1,cate2
0,건들바위,문화적,기념물
1,공룡공원,자연적,테마공원
2,구암서원 (영남선비문화수련원) 숭현사,문화적,문화재
3,국채보상운동기념공원,자연적,테마공원
4,김광석 길,위락적,도보코스
5,꽃보라동산,자연적,근린공원
6,달구벌대종,문화적,문화재
7,대구 삼성 창조 캠퍼스,산업적,복합문화공간
8,대구도시철도,사회적,철도
9,대구제일교회,사회적,개신교


In [44]:
df_code.groupby('cate1').count()

,name,cate2
cate1,,
문화적,3,3
사회적,4,4
산업적,1,1
위락적,5,5
자연적,9,9


In [45]:
df_final = pd.merge(df, df_code)
df_final.head()

,name,date,reviews,stars,lng,lat,cate1,cate2
0,공룡공원,2020-08,굿,3,128.603395,35.829925,자연적,테마공원
1,공룡공원,2020-08,굿,3,128.603395,35.829925,자연적,테마공원
2,국채보상운동기념공원,2020-08,좋아요,5,128.602750,35.869007,자연적,테마공원
3,국채보상운동기념공원,2020-06,주차장 이용했습니다아,5,128.602750,35.869007,자연적,테마공원
4,국채보상운동기념공원,2020-02,대구를 상징하는 좋아요,3,128.602750,35.869007,자연적,테마공원


In [46]:
df_final.groupby('cate1').count()

,name,date,reviews,stars,lng,lat,cate2
cate1,,,,,,,
문화적,45,45,45,45,45,45,45
사회적,1223,1223,1223,1223,1223,1223,1223
산업적,202,202,202,202,202,202,202
위락적,1607,1607,1607,1607,1607,1607,1607
자연적,434,434,434,434,434,434,434


In [52]:
df_final.to_csv('review_data(utf-8).csv')